In [3]:
import pandas as pd
import numpy as np
import sys
from statsmodels.formula.api import ols

In [227]:
def fmp_dividend(code):
    try:
        api_key = '**************'
        url = 'https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/{}?apikey={}'.format(code, api_key)
        dividend_dict = pd.read_json(url).iloc[:,1]
        dividend =pd.DataFrame([dividend_dict[0]]).loc[:,['date', 'adjDividend']]
        for i in range(1,len(dividend_dict)):
            temp2 = pd.DataFrame([dividend_dict[i]]).loc[:,['date', 'adjDividend']]
            dividend = pd.concat([dividend,temp2])
        return dividend
    except :
        print('{} does not pay a dividend'.format(code))
        dividend = 0
    return dividend
    #columns = ['date', 'adjDividend']

In [228]:
fmp_dividend('AAPL')

,date,adjDividend
0,2022-05-06,0.230000
0,2022-02-04,0.220000
0,2021-11-05,0.220000
0,2021-08-06,0.220000
0,2021-05-07,0.220000
...,...,...
0,1988-05-16,0.000714
0,1988-02-12,0.000714
0,1987-11-17,0.000714
0,1987-08-10,0.000536


In [234]:
def fmp_retention_ratio(code):
    #Payout ratio is dividend per share paid to shareholders relative to earnings per share based on GAAP principles
    api_key = '**************'
    url = 'https://financialmodelingprep.com/api/v3/income-statement/{}?&apikey={}'.format(code, api_key)
    pd_is = pd.read_json(url)
    EPS = pd_is[['date', 'epsdiluted']]
    EPS['date'] = pd.to_datetime(EPS['date'], errors='coerce')
    dividend = fmp_dividend(code)
    if type(dividend)==pd.DataFrame:
        payout = []
        for i in range(min(EPS.shape[0], dividend.shape[0])):
            p = 0
            epsyear = EPS['date'].dt.year.iloc[p]
            divyear = pd.to_datetime(dividend['date'], errors='coerce').dt.year.iloc[i]
            if divyear >= epsyear:
                payout.append(dividend['adjDividend'].iloc[i]*4 / EPS['epsdiluted'].iloc[p])
            elif divyear < epsyear:
                try:
                    payout.append(dividend['adjDividend'].iloc[i] / EPS['epsdiluted'].iloc[p+1])
                    p += 1
                except:
                    break
        return [1-x for x in payout]
    else:
        return [1]*EPS.shape[0]
    #output is a list of each year


In [235]:
fmp_retention_ratio('AAPL')

C:\Users\jky93\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


[0.8360071301247771,
 0.8431372549019608,
 0.8431372549019608,
 0.8431372549019608,
 0.8431372549019608,
 0.8538324420677361,
 0.9375,
 0.9375,
 0.9375,
 0.9413109756097561,
 0.9413109756097561,
 0.9413109756097561,
 0.9413109756097561,
 0.944359756097561,
 0.944359756097561,
 0.944359756097561,
 0.944359756097561,
 0.9519817073170732,
 0.9519817073170732,
 0.9519817073170732,
 0.9519817073170732,
 0.9565548780487805,
 0.9565548780487805,
 0.9565548780487805,
 0.9565548780487805,
 0.9603658536585366,
 0.9603658536585366,
 0.9603658536585366,
 0.9603658536585366,
 0.9641768292682927,
 0.9641768292682927,
 0.9641768292682927,
 0.9641768292682927,
 0.9667899390243903,
 0.9667899390243903,
 0.9667899390243903,
 0.9667899390243903]

In [254]:
def fmp_retention_ratio2(code):
    api_key = '**************'
    url = 'https://financialmodelingprep.com/api/v3/key-metrics/{}?apikey={}'.format(code, api_key)
    pd_metric = pd.read_json(url)
    payout = pd_metric[['date', 'payoutRatio']]
    payout['retentionratio'] = payout[['payoutRatio']].apply(lambda x: 1-x)
    return payout

In [255]:
def fmp_growth_rate(code):
    # return fmp_retention_ratio(code) *fmp_ROE(code)
    return fmp_retention_ratio2(code)['retentionratio'] *fmp_ROE(code)

In [260]:
fmp_ROE('AAPL')

0     1.500713
1     0.878664
2     0.610645
3     0.555601
4     0.360702
5     0.356237
6     0.447355
7     0.354200
8     0.299776
9     0.353041
10    0.338341
11    0.293214
12    0.204944
13    0.229862
14    0.240573
15    0.199219
16    0.178811
17    0.054374
18    0.016339
19    0.015873
20   -0.006378
21    0.191381
22    0.193621
23    0.188185
24   -0.870833
25   -0.396501
26    0.146156
27    0.130146
28    0.042736
29    0.242480
30    0.175355
31    0.328242
32    0.305580
33    0.398944
34    0.260012
35    0.221870
36    0.111172
dtype: float64

In [259]:
fmp_growth_rate('AAPL')

C:\Users\jky93\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0     1.271406
1     0.663157
2     0.454613
3     0.427627
4     0.265444
5     0.261499
6     0.350492
7     0.254458
8     0.214271
9     0.331994
10    0.338341
11    0.293214
12    0.204944
13    0.229862
14    0.240573
15    0.199219
16    0.178811
17    0.054374
18    0.016339
19    0.015873
20   -0.006378
21    0.191381
22    0.193621
23    0.188185
24   -0.870833
25   -0.403304
26    0.126163
27    0.106410
28    0.015298
29    0.216330
30    0.143375
31    0.291056
32    0.271724
33    0.359049
34         NaN
35         NaN
36         NaN
dtype: float64

In [258]:
def fmp_ROE(code):
    api_key = '**************'
    url = 'https://financialmodelingprep.com/api/v3/income-statement/{}?limit=120&apikey={}'.format(code, api_key)
    pd_is = pd.read_json(url)
    url2 = 'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{}?limit=120&apikey={}'.format(code, api_key)
    pd_bs = pd.read_json(url2)
    total_equity = pd_bs[['date', 'totalEquity']]
    net_income = pd_is[['date', 'netIncome']]
    ROE =  net_income['netIncome']/total_equity['totalEquity']
    return ROE

In [369]:
fmp_ROE('AAPL')

0    1.500713
1    0.878664
2    0.610645
3    0.555601
4    0.360702
dtype: float64

In [359]:
def required_return(code):
    api_key = '**************'
    url_code = 'https://financialmodelingprep.com/api/v3/historical-price-full/{}?serietype=line&apikey={}'.format(code, api_key)
    pd_code = pd.read_json(url_code)
    url_market = 'https://financialmodelingprep.com/api/v3/historical-price-full/%5EGSPC?apikey={}'.format(api_key)
    pd_market = pd.read_json(url_market)
    
    price = pd.DataFrame()
    price['date'] = pd_code['historical'].map(lambda x: x['date'])
    price['close']  = pd_code['historical'].map(lambda x: x['close'])
    price['mkt_close']  = pd_market['historical'].map(lambda x: x['close'])
    price['return'] = ((price['close'] - price['close'].shift(-1))/price['close'].shift(-1))
    price['mkt_return'] = ((price['mkt_close'] - price['mkt_close'].shift(-1))/price['mkt_close'].shift(-1))
    price.dropna(inplace=True)
    
    riskfree = pd.read_csv('C:/Users/jky93/moggle_labs/portdoctor/US treasury-10y.csv', encoding='CP949')
    riskfree.columns = ['date', 'treasury-10y']
    riskfree['treasury-10y'] = riskfree['treasury-10y'].map(lambda x: float(x)*0.01 if x != '.' else None)
    riskfree.replace('.', np.NaN).fillna(method ='ffill')
    data = pd.merge(price, riskfree)

    df = pd.DataFrame([(data['return']-data['treasury-10y']), (data['mkt_return']-data['treasury-10y'])])      
    df = df.T
    df.columns = ['security', 'mkt']
    model = ols(formula='security ~ mkt', data = df)
    beta = model.params['mkt']

    required = data.loc[0, 'treasury-10y'] + beta*(data.loc[0, 'mkt_return'] - data.loc[0, 'treasury-10y'])
    return required

In [366]:
fmp_dividend('MSFT')['adjDividend'].values[0]

0.62

In [380]:
def fmp_growth_rate(code):
    return fmp_retention_ratio(code) *fmp_ROE(code)

In [213]:
growth_rate('AAPL')

C:\Users\jky93\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0    0.557073
1    0.959568
2    1.380733
3    1.517522
4    2.337491
dtype: float64

In [367]:
growth_rate('AAPL')[0]

C:\Users\jky93\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


0.5570731922219284

In [ ]:
def RIM(code):
    # Assumption : No dividend firm, Earning and book equity grow at a constant rate of g
    # Required EPS = Book equity per share of t-1 * required rate
    # RI = EPS - Required EPS
    # [현재 EPS * (1+g) - 현재 BookEquity * g] / (k-g)
    api_key = '**************'
    url = 'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{}?limit=120&apikey={}'.format(code, api_key)
    pd_bs = pd.read_json(url)
    total_equity = pd_bs[['date', 'totalEquity']]
    url2 = 'https://financialmodelingprep.com/api/v3/income-statement/{}?&apikey={}'.format(code, api_key)
    pd_is = pd.read_json(url2)
    EPS = pd_is[['date', 'epsdiluted']]
    EPS['date'] = pd.to_datetime(EPS['date'], errors='coerce')
    
    return 

In [ ]:
code = 'AAPL'
api_key ='**************'
url = 'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{}?limit=120&apikey={}'.format(code, api_key)
pd_bs = pd.read_json(url)
total_equity = pd_bs[['date', 'totalEquity']]
url2 = 'https://financialmodelingprep.com/api/v3/income-statement/{}?&apikey={}'.format(code, api_key)
pd_is = pd.read_json(url2)
EPS = pd_is[['date', 'epsdiluted']]
EPS['date'] = pd.to_datetime(EPS['date'], errors='coerce')
current_Equity = total_equity.loc[0, 'totalEquity']
current_EPS = EPS.loc[0, 'epsdiluted']

growth_rate = fmp_growth_rate(code)
required_rate = 0.3
# required_rate = fmp_required_return(code)
RI = EPS - total_equity * required_rate
PV = (current_EPS*(1+g) - current_Equity * growth_rate)/(required_rate - growth_rate)

In [384]:
EPS

,date,epsdiluted
0,2021-09-25,5.6100
1,2020-09-26,3.2800
2,2019-09-28,2.9725
3,2018-09-29,2.9775
4,2017-09-30,2.3025


In [385]:
total_equity

,date,totalEquity
0,2021-09-25,63090000000
1,2020-09-26,65339000000
2,2019-09-28,90488000000
3,2018-09-29,107147000000
4,2017-09-30,134047000000


In [383]:
EPS - total_equity

,date,epsdiluted,totalEquity
0,0 days,NaN,NaN
1,0 days,NaN,NaN
2,0 days,NaN,NaN
3,0 days,NaN,NaN
4,0 days,NaN,NaN


### DCF

In [10]:
from urllib.request import urlopen
import certifi
import json
code = 'AAPL'
api_key = '**************'
url = 'https://financialmodelingprep.com/api/v4/company-outlook?symbol={}&apikey={}'.format(code, api_key)
response = urlopen(url, cafile=certifi.where())
data = response.read().decode("utf-8")
result = json.loads(data)


C:\Users\jky93\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  import sys


In [49]:
#dict_keys(['profile', 'metrics', 'ratios', 'insideTrades', 'keyExecutives', 'splitHistory', 'stockDividend', 'stockNews', 'rating', 'financialsAnnual', 'financialsQuarter'])
date = []
for i in range(len(result['stockDividend'])):
    date.append(result['stockDividend'][i]['date'])
date

['2022-05-06', '2022-02-04', '2021-11-05', '2021-08-06', '2021-05-07']

In [52]:
result['financialsQuarter'].keys() #dict_keys(['income', 'balance', 'cash'])

dict_keys(['income', 'balance', 'cash'])

In [58]:
result['metrics'].keys()   #dict_keys(['dividendYielTTM', 'volume', 'yearHigh', 'yearLow'])

dict_keys(['dividendYielTTM', 'volume', 'yearHigh', 'yearLow'])

In [64]:
a=result['financialsQuarter']['cash']

date = []
for i in range(len(a)):
    date.append(a[i]['date'])
date

['2022-03-26', '2021-12-25', '2021-09-25', '2021-06-26', '2021-03-27']

In [132]:
def FCF(code):
    api_key = '**************'
    url = 'https://financialmodelingprep.com/api/v3/income-statement/{}?limit=120&apikey={}'.format(code, api_key)
    pd_is = pd.read_json(url)
    pd_fcf = pd_is[['date', 'ebitda', 'netIncome', 'interestExpense', 'interestIncome', 'depreciationAndAmortization', 'operatingIncome', 'operatingExpenses', 'researchAndDevelopmentExpenses', 'otherExpenses']]
    netoperating = (pd_ebitda['operatingExpenses'] - pd_ebitda['operatingIncome']).diff(-1).fillna(0)
    pd_fcf['netoperating'] = netoperating
    pd_fcf['fcf'] = pd_ebitda['netIncome'] + pd_ebitda['interestExpense'] - pd_ebitda['interestIncome'] +pd_ebitda['depreciationAndAmortization'] - pd_ebitda['netoperating'] - pd_ebitda['researchAndDevelopmentExpenses'] + pd_ebitda['otherExpenses']
    return  pd_fcf[['date', 'fcf']]

In [133]:
fcf = FCF('AAPL')
fcf

C:\Users\jky93\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\jky93\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,fcf
0,2021-09-25,1.212940e+11
1,2020-09-26,4.697700e+10
2,2019-09-28,3.971200e+10
3,2018-09-29,5.920700e+10
4,2017-09-30,4.276600e+10
5,2016-09-24,3.055500e+10
6,2015-09-26,6.876100e+10
7,2014-09-27,4.077900e+10
8,2013-09-28,2.971300e+10
9,2012-09-29,5.859900e+10


In [136]:
code = 'AAPL'
api_key = '**************'
url = 'https://financialmodelingprep.com/api/v4/advanced_levered_discounted_cash_flow?symbol={}&apikey={}'.format(code, api_key)
pd_dcf = pd.read_json(url)

In [137]:
pd_dcf.columns

Index(['year', 'symbol', 'revenue', 'revenuePercentage', 'capitalExpenditure',
       'capitalExpenditurePercentage', 'price', 'beta',
       'dilutedSharesOutstanding', 'costofDebt', 'taxRate',
       'afterTaxCostOfDebt', 'riskFreeRate', 'marketRiskPremium',
       'costOfEquity', 'totalDebt', 'totalEquity', 'totalCapital',
       'debtWeighting', 'equityWeighting', 'wacc', 'operatingCashFlow',
       'pvLfcf', 'sumPvLfcf', 'longTermGrowthRate', 'freeCashFlow',
       'terminalValue', 'presentTerminalValue', 'enterpriseValue', 'netDebt',
       'equityValue', 'equityValuePerShare', 'freeCashFlowT1',
       'operatingCashFlowPercentage'],
      dtype='object')

In [216]:
def unleveled_DCF_fmp_calculation(code, periods=4): 
    #periods : years into the future, FMP provides 4-years ahead prediction.
    api_key = '**************'
    url = 'https://financialmodelingprep.com/api/v4/advanced_discounted_cash_flow?symbol={}&apikey={}'.format(code, api_key)
    pd_dcf = pd.read_json(url)
    wacc = np.mean(pd_dcf['wacc'].iloc[0:5]) *0.01
    ufcf = pd_dcf['ufcf']
    growth_rate = np.mean(pd_dcf['longTermGrowthRate'].iloc[0:5]) *0.01
    cash = pd_dcf['totalCash'][periods]
    netDebt = pd_dcf['netDebt'][periods]
    TV = ufcf[0]*(1+growth_rate)  / (wacc - growth_rate)
    NPV_TV = TV / (1+wacc)**(periods)
    NPV_FCF = 0
    for i in range(periods):
        NPV_FCF += ufcf[i] / (1+wacc)**(periods-i)
    EV = NPV_TV + NPV_FCF + cash
    MV = EV - netDebt
    shares = pd_dcf['dilutedSharesOutstanding'][periods]
    return {'mv':MV/shares, 'EV':EV/shares, 'NPV_TV':NPV_TV/shares, 'NPV_FCF':NPV_FCF/shares, 'netDebt':netDebt/shares, 'cash':cash/shares, 'wacc':wacc, 'growth':growth_rate}   

In [217]:
unleveled_DCF_fmp_calculation('AAPL')

{'mv': 131.23424563628967,
 'EV': 136.55766252768316,
 'NPV_TV': 107.37267472648404,
 'NPV_FCF': 25.470828469284132,
 'netDebt': 5.323416891393499,
 'cash': 3.714159331915006,
 'wacc': 0.0844,
 'growth': 0.02}

In [218]:
def unleveled_DCF_fmp_value(code, periods=4): 
    api_key = '**************'
    url = 'https://financialmodelingprep.com/api/v4/advanced_discounted_cash_flow?symbol={}&apikey={}'.format(code, api_key)
    pd_dcf = pd.read_json(url).iloc[0,:]
    shares = pd_dcf['dilutedSharesOutstanding']
    NPV_TV = pd_dcf['presentTerminalValue']
    NPV_FCF = pd_dcf['sumPvUfcf']
    netDebt = pd.read_json(url).iloc[periods,:]['netDebt']
    wacc = pd_dcf['wacc'] * 0.01
    mv = pd_dcf['equityValuePerShare']
    ev = pd_dcf['enterpriseValue']
    cash = pd.read_json(url).iloc[periods,:]['totalCash']
    return {'mv':mv, 'EV':ev/shares, 'NPV_TV':NPV_TV/shares, 'NPV_FCF':NPV_FCF/shares, 'netDebt':netDebt/shares, 'cash':cash/shares, 'wacc':wacc}


In [219]:
unleveled_DCF_fmp_value('AAPL')

{'mv': 122.75,
 'EV': 128.0717092046687,
 'NPV_TV': 99.01574510878208,
 'NPV_FCF': 29.055963502939846,
 'netDebt': 5.323416891393499,
 'cash': 3.714159331915006,
 'wacc': 0.0844}

In [192]:
def DCF_fmp(code):
    api_key = '**************'
    url = 'https://financialmodelingprep.com/api/v3/historical-discounted-cash-flow-statement/{}?apikey={}'.format(code, api_key)   
    pd_dcf = pd.read_json(url)
    return pd_dcf

In [193]:
DCF_fmp('AAPL')

,symbol,date,price,dcf
0,AAPL,2021-09-25,148.64,150.73
1,AAPL,2020-09-26,115.05,117.24
2,AAPL,2019-09-28,62.26,64.91
3,AAPL,2018-09-29,53.06,54.37
4,AAPL,2017-09-30,41.68,42.65
5,AAPL,2016-09-24,29.41,30.35
6,AAPL,2015-09-26,28.82,29.74
7,AAPL,2014-09-27,26.28,26.85
8,AAPL,2013-09-28,18.92,19.47
9,AAPL,2012-09-29,21.57,21.98


In [151]:
code = 'GOOGL'
api_key = '**************'
url = 'https://financialmodelingprep.com/api/v3/historical-discounted-cash-flow-statement/{}?apikey={}'.format(code, api_key)   
pd_dcf = pd.read_json(url)
pd_dcf

,symbol,date,price,dcf
0,GOOGL,2021-12-31,2667.02,2698.39
1,GOOGL,2020-12-31,1827.36,1866.23
2,GOOGL,2019-12-31,1454.25,1480.96
3,GOOGL,2018-12-31,1097.99,1122.02
4,GOOGL,2017-12-31,1177.37,1192.83
5,GOOGL,2016-12-31,823.83,842.61
6,GOOGL,2015-12-31,761.35,785.52
7,GOOGL,2014-12-31,537.55,564.69
8,GOOGL,2013-12-31,568.26,596.68
9,GOOGL,2012-12-31,377.29,399.90
